# Day 13: Capstone Project
## Comprehensive Multi-Asset Margin Model
This notebook integrates all concepts to build a production-ready margin allocation model.

In [ ]:
from mosek.fusion import *
import numpy as np
import pandas as pd

def capstone_model(returns: pd.DataFrame, initial_weights: np.ndarray, target_return: float, var_limit: float, lambda_tc: float) -> np.ndarray:
    n = returns.shape[1]
    mu = returns.mean().values
    Sigma = returns.cov().values

    with Model("Capstone") as M:
        w = M.variable("w", n, Domain.unbounded())
        t = M.variable("t", n, Domain.unbounded())  # Transaction cost variable

        # Objective: Minimize variance + transaction costs
        M.objective("obj", ObjectiveSense.Minimize, Expr.add(Expr.dot(w, Expr.mul(Sigma, w)), Expr.mul(lambda_tc, Expr.sum(t))))

        # Constraints
        M.constraint("return", Expr.dot(mu, w), Domain.greaterThan(target_return))
        M.constraint("var", Expr.dot(w, Expr.mul(Sigma, w)), Domain.lessThan(var_limit**2))
        M.constraint("tc_pos", Expr.sub(w, initial_weights), Domain.lessThan(t))
        M.constraint("tc_neg", Expr.sub(initial_weights, w), Domain.lessThan(t))

        M.solve()
        return w.level()

# Example usage
np.random.seed(42)
returns = pd.DataFrame(np.random.normal(0, 0.01, (252, 5)))
initial_weights = np.ones(5) / 5
optimal_weights = capstone_model(returns, initial_weights, 0.05, 0.1, 0.001)
print(f'Optimal Weights: {optimal_weights}')

## Documentation
- **Technical**: Model architecture, code documentation.
- **Business**: Impact analysis, user guide.
- **Regulatory**: Compliance with FINRA, Basel III.

## Automation
- **CI/CD**: Automated testing and deployment pipelines.
- **Monitoring**: Real-time performance tracking.